# Elements of Data Science: A First Course 

# COMS W4995 007 2018 3


## Week 11 :  Recommendation Engines (cont.), Data Management, Webscraping and Review

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Today

- Recommendation Engines (continued from week 10)
    - Content Based
    - Collaborative Filtering
- Data Management
    - Strucured vs. Unstructured
    - Relational vs. NoSQL
- Review

# Recommendation Engines

Continued from week 10

# Managing Data

- Structured

- Unstructured

- Semi-structured

# Databases

- Flatfile
- Relational
- NoSQL

## Flatfile

- csv

## Relational / RDBMS / SQL

- sqlite
- Postgresql
- Mysql
- MariaDB
- Microsoft SQL Server
- Oracle

### Database Normalization

- reduce data redundancy
- improve data integrity

<img src='http://www.ecommerce-digest.com/images/normalization.jpg'>

### Basic SQL Commands

- SELECT

    <pre>
    SELECT * FROM trips;
    SELECT id, fare_amount FROM trips;</pre>

- WHERE

    <pre>
    SELECT * 
    FROM trips 
    WHERE vendor_id = 1 AND fare_amount > 10.0;</pre>

- LIMIT

    <pre>
    SELECT * 
    FROM trips 
    WHERE vendor_id = 1 AND fare_amount > 10.0
    LIMIT 10;</pre>

- DISTINCT

    <pre>
    SELECT DISTINCT vendor_id FROM trips</pre>

- COUNT

    <pre>
    SELECT COUNT(*) FROM trips</pre>

- ORDER BY

    <pre>
    SELECT fare_amount, passenger_count 
    FROM trips 
    ORDER BY fare_amount DESC;</pre>

- JOIN

    <pre>
    SELECT t.fare_amount, tzp.borough 
    FROM trips AS t
    LEFT JOIN taxi_zones AS tz ON tzp.locationid = t.pickup_location_id
    LIMIT 10</pre>

- Putting it all together

```
    SELECT 
        t.fare_amount
        ,t.trip_distance
        ,tzp.borough as pickup_borough
        ,tzd.borough as dropoff_borough
    FROM
        trips AS t
        LEFT JOIN taxi_zones AS tzp ON tzp.locationid = t.pickup_location_id
        LEFT JOIN taxi_zones AS tzd ON tzd.locationid = t.dropoff_location_id
    WHERE 
        t.fair_amount > 0 
        AND tzp.borough = 'Manhattan'
    ORDER BY
        t.trip_distance DESC
    LIMIT 10
```

## sqlalchemy

In [64]:
import sqlalchemy

db = sqlalchemy.create_engine('postgres:///nyc-taxi-data?host=/var/run/postgresql')

In [71]:
# perform the query
q = db.execute('SELECT * FROM trips LIMIT 2')

In [72]:
# what is q?
print(type(q))

<class 'sqlalchemy.engine.result.ResultProxy'>


In [73]:
# what are the column names in the return set?
print(q._metadata.keys)

['id', 'cab_type_id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'rate_code_id', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'pickup_nyct2010_gid', 'dropoff_nyct2010_gid', 'pickup_location_id', 'dropoff_location_id', 'pickup', 'dropoff']


In [74]:
# and getting the returned data
for r in q:
    print(r);print()

(1, 1, '1', datetime.datetime(2016, 10, 1, 0, 0), datetime.datetime(2016, 10, 1, 0, 10, 45), 'N', 1, None, None, None, None, 3, Decimal('3.40'), Decimal('13'), Decimal('0.5'), Decimal('0.5'), Decimal('3.55'), Decimal('0'), None, Decimal('0.3'), Decimal('17.85'), '1', None, None, None, 80, 33, None, None)

(2, 1, '2', datetime.datetime(2016, 10, 1, 0, 0), datetime.datetime(2016, 10, 1, 0, 15, 51), 'N', 1, None, None, None, None, 5, Decimal('4.17'), Decimal('15'), Decimal('0.5'), Decimal('0.5'), Decimal('2'), Decimal('0'), None, Decimal('0.3'), Decimal('18.3'), '1', None, None, None, 262, 107, None, None)



In [75]:
# a more complex query
sql = """
SELECT 
    t.id
    ,t.fare_amount
    ,t.trip_distance
    ,tzp.borough as pickup_borough
    ,tzd.borough as dropoff_borough
FROM
    trips AS t
    LEFT JOIN taxi_zones AS tzp ON tzp.locationid = t.pickup_location_id
    LEFT JOIN taxi_zones AS tzd ON tzd.locationid = t.dropoff_location_id
WHERE 
    t.fare_amount > 0 
    AND tzp.borough = 'Manhattan'
ORDER BY
    t.trip_distance DESC
LIMIT 10
"""

In [76]:
# we can pull directly into a dataframe using pandas
df = pd.read_sql(sql,con=db, index_col='id')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 4874540 to 10154756
Data columns (total 4 columns):
fare_amount        10 non-null float64
trip_distance      10 non-null float64
pickup_borough     10 non-null object
dropoff_borough    10 non-null object
dtypes: float64(2), object(2)
memory usage: 400.0+ bytes
None


## NoSQL

- key-value (Redis, Berkeley DB)
- document store (MongoDB, DocumentDB)
- wide column (Cassandra, HBase, DynamoDB
- graph (Neo4j, Giraph)

### Basic Mongo

- db

- collection

- find(filter,projection)

In [8]:
import pymongo

In [9]:
# start up our client, defaults to the local machine
mdb = pymongo.MongoClient()

In [10]:
# get a connection to a database
db = mdb.twitter_db

In [11]:
# get a connection to a collection in that database
coll = db.twitter_collection

In [12]:
# get one record
coll.find_one()

{'_id': ObjectId('59c95e2c2471847a9783c400'),
 'created_at': 'Mon Sep 25 19:51:08 +0000 2017',
 'id': 912404120484511749,
 'id_str': '912404120484511749',
 'text': 'RT @YarmolukDan: Waste Management Just Got Cleaner and More Efficient https://t.co/HtaXzfxbrA #DataScience #DataScientist #BigData #AI #IoT…',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 912391257430794241,
  'id_str': '912391257430794241',
  'name': 'Roxane Wattenbarger',
  'screen_name': 'roxanewattenba6',
  'location': None,
  'url': None,
  'description': 'l',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 0,
  'friends_count': 39,
  'listed_count': 0,
  'favourites_count': 17,
  'statuses_count': 33,
  'created_at': 'Mon

In [13]:
# get one record and pull into a dataframe
df = pd.DataFrame([coll.find_one()])

In [14]:
df

,_id,contributors,coordinates,created_at,entities,favorite_count,favorited,filter_level,geo,id,...,quote_count,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,59c95e2c2471847a9783c400,None,None,Mon Sep 25 19:51:08 +0000 2017,"{'hashtags': [{'text': 'DataScience', 'indices...",0,False,low,None,912404120484511749,...,0,0,0,False,{'created_at': 'Mon Sep 25 19:00:38 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @YarmolukDan: Waste Management Just Got Cle...,1506369068629,False,"{'id': 912391257430794241, 'id_str': '91239125..."


In [15]:
# iterate through a list saved in a dataframe
hts = df.entities[0]['hashtags']
for ht in hts:
    print(ht['text'])

DataScience
DataScientist
BigData
AI
IoT


In [77]:
# a more complicated query: find all usernames and entities for tweets with hashtag AI
q = coll.find({'entities.hashtags.text':'AI'},{'user.screen_name':1,'entities':1})
for r in q:
    print(r['user']['screen_name'])
    print([x['text'] for x in r['entities']['hashtags']])

roxanewattenba6
['DataScience', 'DataScientist', 'BigData', 'AI', 'IoT']
sawney_patience
['DataScience', 'DataScientist', 'BigData', 'AI', 'IoT']
jackverr54
['MachineLearning', 'BigData', 'DataScience', 'AI', 'RStats', 'RLang', 'Statistics']
AvvRossello
['Chatbots', 'Infographic', 'AI', 'DeepLearning', 'Startup', 'IoT', 'BigData', 'Analytics', 'DataScience', 'Fintech']
Follow2bFollowd
['AI', 'MachineLearning', 'Fintech', 'insurtech', 'ML', 'DL', 'tech']
Follow2bFollowd
['AI', 'MachineLearning', 'BigData', 'Fintech', 'ML', 'Banking', 'tech']
Calcaware
['BigData', 'DataScience', 'MachineLearning', 'AI', 'DeepLearning', 'fintech']


## Webscraping

#### Basic html page

```
<!DOCTYPE html>
<html>
<head>
    <title>Web Page!</title>
    <style>
        body {background-color: powderblue;}
        h1   {color: blue;}
        p    {color: red;}
    </style>
    <link rel="stylesheet" href="styles.css">
    <script>
        document.getElementById("demo").innerHTML = "Hello JavaScript!";
    </script>
</head>
<body>
    <h1>A Very Bold Header</h1>
    <div style="background-color:lightblue">
        <p>This is a paragraph.</p>
    </div>
</body>
</html>
```

### nyc weather history

http://w1.weather.gov/data/obhistory/KNYC.html

In [17]:
knyc_link = 'http://w1.weather.gov/data/obhistory/KNYC.html'

In [18]:
import requests

knyc_page = requests.get(knyc_link)
knyc_page

<Response [200]>

In [19]:
# the first 1000 characters of the page
print(knyc_page.content[:1000])

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">\r\n\t\t\t\t\t\t\t<html><meta name="Author" content="Leon Minton"><head><title>\r\n\t\t\t\t\t\t\tNational Weather Service : Observed Weather for past 3 Days : New York City, Central Park</title>\r\n\t\t\t\t\t\t\t<link rel="STYLESHEET" type="text/css" href="http://www.srh.noaa.gov/weather/images/fcicons/main.css"></head>\r\n\t\t\t\t\t\t\t<body bgcolor="#ffffff" leftmargin="0" topmargin="0" marginwidth="0" marginheight="0" background="/images/weather/fcicons/gray_background.gif">\r\n\t\t\t\t\t\t\t<table cellspacing="0" cellpadding="0" border="0" width="670" background="/images/weather/fcicons/topbanner.jpg">\r\n\t\t\t\t\t\t\t<tr><td align="right" height="19"><a href="http://weather.gov"><span class="nwslink">weather.gov</span></a>&nbsp;&nbsp;&nbsp;</td></tr></table>\r\n\t\t\t\t\t\t\t<table cellspacing="0" cellpadding="0" border="0" width="670"><tr valign="top">\r\n\t\t\t\t\t\t\t<td rowspan="2"><a href="http://www.noaa.gov"><

In [20]:
# need to parse some html!
from bs4 import BeautifulSoup

In [21]:
knyc_soup = BeautifulSoup(knyc_page.content)

In [22]:
# first 1000 characters more legibly
print(knyc_soup.prettify()[:1000])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<html>
 <head>
  <meta content="Leon Minton" name="Author"/>
  <title>
   National Weather Service : Observed Weather for past 3 Days : New York City, Central Park
  </title>
  <link href="http://www.srh.noaa.gov/weather/images/fcicons/main.css" rel="STYLESHEET" type="text/css"/>
 </head>
 <body background="/images/weather/fcicons/gray_background.gif" bgcolor="#ffffff" leftmargin="0" marginheight="0" marginwidth="0" topmargin="0">
  <table background="/images/weather/fcicons/topbanner.jpg" border="0" cellpadding="0" cellspacing="0" width="670">
   <tr>
    <td align="right" height="19">
     <a href="http://weather.gov">
      <span class="nwslink">
       weather.gov
      </span>
     </a>
    </td>
   </tr>
  </table>
  <table border="0" cellpadding="0" cellspacing="0" width="670">
   <tr valign="top">
    <td rowspan="2">
     <a href="http://www.noaa.gov">
      <img alt="NOAA logo - Click to go to the NOAA homepage" b

In [23]:
# print the 4rd table in the page
print(knyc_soup.find_all('table')[3])

<table border="0" cellpadding="2" cellspacing="3" width="670"><tr align="center" bgcolor="#b0c4de"><th rowspan="3" width="17">D<br/>a<br/>t<br/>e</th><th rowspan="3" width="32">Time<br/>(est)</th>
<th rowspan="3" width="80">Wind<br/>(mph)</th><th rowspan="3" width="40">Vis.<br/>(mi.)</th><th rowspan="3" width="80">Weather</th><th rowspan="3" width="65">Sky Cond.</th>
<th colspan="4">Temperature (ºF)</th><th rowspan="3" width="65">Relative<br/>Humidity</th><th rowspan="3" width="80">Wind<br/>Chill<br/>(°F)</th><th rowspan="3" width="80">Heat<br/>Index<br/>(°F)</th><th colspan="2">Pressure</th><th colspan="3">Precipitation (in.)</th></tr>
<tr align="center" bgcolor="#b0c4de"><th rowspan="2" width="45">Air</th><th rowspan="2" width="26">Dwpt</th><th colspan="2">6 hour</th>
<th rowspan="2" width="40">altimeter<br/>(in)</th><th rowspan="2" width="40">sea level<br/>(mb)</th><th rowspan="2" width="24">1 hr</th>
<th rowspan="2" width="24">3 hr</th><th rowspan="2" width="30">6 hr</th></tr>
<tr 

In [24]:
# extract data from the 4th table in the page into a dataframe

data_table = knyc_soup.find_all('table')[3]

table_rows = data_table.find_all('tr') # get rows from table

data = []
for idx,tr in enumerate(table_rows):
    if idx < 3 :                       # skip header rows
        continue
    td = tr.find_all('td')             # get table cells
    row = [elem.text for elem in td]   # pull text from cells
    data.append(row)                   # add to dataset
    
pd.DataFrame(data).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,03,13:51,NA,10.00,A Few Clouds,FEW050,53,33,,,47%,NA,NA,29.66,1003.7,,,
1,03,12:51,NA,10.00,Overcast,OVC049,52,34,55,52,50%,NA,NA,29.67,1003.8,,,
2,03,11:51,NA,10.00,Overcast,OVC037,52,37,,,57%,NA,NA,29.67,1003.9,,,
3,03,10:51,NA,10.00,A Few Clouds,FEW037,54,39,,,57%,NA,NA,29.67,1003.9,,,
4,03,09:51,NA,10.00,Fair,CLR,54,41,,,62%,NA,NA,29.67,1003.8,,,


#### central park weather history summary
https://www.wunderground.com/history/daily/us/ny/new-york-city/KNYC/date/2018-12-3?cm_ven=localwx_history

In [25]:
wu_link = 'https://www.wunderground.com/history/daily/us/ny/new-york-city/KNYC/date/2018-12-3?cm_ven=localwx_history'

In [26]:
# get the page
wu_page = requests.get(wu_link)
wu_page

<Response [200]>

In [28]:
wu_soup = BeautifulSoup(wu_page.content)

In [30]:
print(wu_soup.prettify()[:1000])

<!DOCTYPE html>
<html>
 <head>
  <title>
   Central Park, NY History | Weather Underground
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="general" name="rating"/>
  <meta content="no-referrer-when-downgrade" name="referrer"/>
  <meta content="app-id=486154808, affiliate-data=at=1010lrYB&amp;ct=website_wu" name="apple-itunes-app"/>
  <meta content="325331260891611" name="fb_app_id"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="fb_channel_url"/>
  <meta content="Weather Underground" property="og:site_name"/>
  <meta content="article" property="og:type"/>
  <meta content="Weather Underground provides local &amp; long range weather forecasts, weather reports, maps &amp; tropical weather conditions for locations worldwide." name="description"/>
  <meta content="false" name="wui-member-logged-in"/>
 

In [29]:
# the table we want doesn't exist! culprit: javascript
wu_soup.find_all('div',class_='tablesaw-sortable')

[]

In [31]:
# get the text from the page
wu_text = wu_soup.get_text()

# clean up the whitespace
import re
wu_text = re.sub(r'\n+','\n',text.strip())
print(text[:1000])

Central Park, NY History | Weather Underground
  //<![CDATA[
  window.webpackManifest = {"0":"city-history-module.271410e14d01eca31253.js","1":"video-module.94791ee0736f33f5568c.js","2":"health-module.76922905d5d1554bc3c6.js","3":"hurricane-module.b45b656d6deabd837533.js","4":"city-today-module.127cf5745059112cf3ac.js","5":"city-ten-day-module.7ed9ac78f02c8aa5e3d4.js","6":"city-hourly-module.bc5753f0cb1d1866d6d8.js","7":"precipitation-module.df41624c9e83a1f6dd81.js","8":"city-history-calendar-module.2988780f63ffe141a666.js","9":"city-severe-module.70817f8d824b13fbde08.js","10":"article-page-module.7c507107e75698855672.js","11":"page-module.0d95aaa43f5a7a267c2c.js","12":"member-mydevices-module.6fcf4e3f03eb5d5965ab.js","13":"landing-purpleair-module.72a1382f54cb1a732850.js","14":"test-module.c6d3df57d62511f4345f.js","15":"hurricane-storm-module.4fa5ad48c4b52827cfe2.js","16":"wundermap-module.e458239922d9a2289e79.js","17":"homepage-module.483ec2af1ec142a28e05.js","18":"cat-six-article-mo

### Need to actually render page to process scripts!

In [32]:
# need to install chromedriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)

In [33]:
# this will actually render the page
driver.get(wu_link)

In [47]:
# two ways to find the table we want
wu_table = driver.find_element_by_class_name('city-history-observation')
#wu_table = driver.find_element_by_id('history-observation-table')

In [48]:
# text in the table
wu_table.text

'Daily Observations\nTime Temperature Dew Point Humidity Wind Wind Speed Wind Gust Pressure Precip.\n12:51 AM\n55 F 52 F 89 %\n0 mph 0 mph 29.4 in 0.0 in\n1:51 AM\n55 F 52 F 89 %\n0 mph 0 mph 29.4 in 0.0 in\n2:51 AM\n55 F 53 F 93 %\n0 mph 0 mph 29.4 in 0.0 in\n3:38 AM\n54 F 53 F 97 %\n0 mph 0 mph 29.4 in 0.0 in\n3:51 AM\n54 F 53 F 97 %\n0 mph 0 mph 29.4 in 0.0 in\n4:51 AM\n54 F 53 F 97 %\n0 mph 0 mph 29.4 in 0.0 in\n5:01 AM\n54 F 53 F 97 %\n0 mph 0 mph 29.4 in 0.0 in\n5:28 AM\n54 F 52 F 93 %\n0 mph 0 mph 29.4 in 0.0 in\n5:51 AM\n54 F 51 F 90 %\n0 mph 0 mph 29.4 in 0.0 in\n6:51 AM\n53 F 49 F 86 %\n0 mph 0 mph 29.4 in 0.0 in\n7:51 AM\n54 F 46 F 75 %\n0 mph 0 mph 29.5 in 0.0 in\n8:51 AM\n54 F 44 F 69 %\n0 mph 0 mph 29.5 in 0.0 in\n9:51 AM\n54 F 41 F 62 %\n0 mph 0 mph 29.5 in 0.0 in\n10:51 AM\n54 F 39 F 57 %\n0 mph 0 mph 29.5 in 0.0 in\n11:51 AM\n52 F 37 F 57 %\n0 mph 0 mph 29.5 in 0.0 in\n12:51 PM\n52 F 34 F 50 %\n0 mph 0 mph 29.5 in 0.0 in\n1:51 PM\n53 F 33 F 47 %\n0 mph 0 mph 29.5 in 0.

In [49]:
# extracting text into a datafram
wu_data = []
for tr in wu_table.find_elements_by_css_selector('tr'):
    tmp_row = []
    for td in tr.find_elements_by_css_selector('td'):
        tmp_row.append(td.text.strip())
    wu_data.append(tmp_row)
df_wu = pd.DataFrame(wu_data)
df_wu.head()

,0,1,2,3,4,5,6,7,8,9,10
0,None,None,None,None,None,None,None,None,None,None,None
1,12:51 AM,55 F,52 F,89 %,,0 mph,0 mph,29.4 in,0.0 in,,
2,1:51 AM,55 F,52 F,89 %,,0 mph,0 mph,29.4 in,0.0 in,,
3,2:51 AM,55 F,53 F,93 %,,0 mph,0 mph,29.4 in,0.0 in,,
4,3:38 AM,54 F,53 F,97 %,,0 mph,0 mph,29.4 in,0.0 in,,


In [52]:
# visualize the rendered table, still missing some stuff, need to debug
wu_table.screenshot('./images/test1.png')

True

<img src='./images/test2.png'>